In [ ]:
import hoss
import os
import time

## Connect to local server
This notebook demonstrates basic operations using a single Hoss server.
For these demo notebooks, it's assumed you have the `admin` role and are running the server locally
on localhost. If using a different server, be sure to change the endpoint in the `.connect()` call.

We start by connecting the the "local" server. 

In [ ]:
server_local = hoss.connect('http://localhost')

In [ ]:
print("Existing Namespaces:")
print(server_local.list_namespaces())

## Create a dataset
First load the default namespace and then create a dataset inside the namespace

In [ ]:
ns = server_local.get_namespace('default')

In [ ]:
ds = ns.create_dataset("simple-test", "A dataset for a simple example")
ds.display()

## Write and read files from the dataset
The client library exposes familiar `pathlib`-like interfaces to interact with object stores
like they were files

In [ ]:
# Object references follow a path building method similar to pathlib
f1 = ds / "my-file.txt"
f1.write_text("Hello, Hoss!")

In [ ]:
# Read/write text easily
print(f1.read_text())

In [ ]:
# Refs have useful properties
print(f1.etag)
print(f1.size_bytes)
print(f1.last_modified)

In [ ]:
f1.write_text("Hello, Hoss!!!!!!!!!")

In [ ]:
# Refs have useful properties
print(f1.etag)
print(f1.size_bytes)
print(f1.last_modified)

In [ ]:
# Can write bytes directly
f2 = ds / "my-file.bin"
f2.write_bytes(b"some data")

In [ ]:
# Read text easily as well
print(f2.read_bytes())

In [ ]:
# You can write from a file easily
filename = os.path.join(os.getcwd(), "example-file.txt")

f3 = ds / "example-files" / "file1.txt"
print(f3.exists())

In [ ]:
f3.write_from(filename)
print(f3.exists())

In [ ]:
print(f3.read_text())

In [ ]:
# A context manager interface is available as well
with f3.open('rt') as fh:
    print(fh.read())

In [ ]:
# It's easy to delete objects too
f3.remove()
print(f3.exists())

## Interact and find with files in a dataset
glob, rglob, and iterdir interfaces are available to iterate through files in a dataset.

For large datasets, this can be useful way to interate and filter.

In [ ]:
# write a bit more data
f1 = ds / "foo0.txt"
f1.write_text("foo0")
f1 = ds / "folder1" / "foo1.txt"
f1.write_text("foo1")
f1 = ds / "folder1" / "foo2.txt"
f1.write_text("foo2")
f1 = ds / "folder1" / "foo3.txt"
f1.write_text("foo3")
f1 = ds / "folder1" / "bar0.txt"
f1.write_text("bar0")

In [ ]:
for f in ds.glob("*"):
    print(f)

In [ ]:
for f in (ds).glob("foo*"):
    print(f)

In [ ]:
for f in (ds).glob("**/foo*"):
    print(f)

In [ ]:
for f in (ds).iterdir():
    print(f)

In [ ]:
for f in (ds / "folder1").iterdir():
    print(f)

## Use metadata to enrich datasets
Metadata key-value pairs can be attached to objects when they are written.

Note, both keys and values must be strings. They are case insensitive and will be converted to lowercase.

In [ ]:
f1 = ds / "with-meta"/ "test1.txt"

f1.write_text("my data file", metadata={'foo': 'bar', 'fizz': 'buzz'})

In [ ]:
print(f1.metadata)

In [ ]:
# If you omit metadata on an object that already has some, it persists automatically
f1.write_text("i changed my file")
print(f1.metadata)
print(f1.read_text())

In [ ]:
# If you set an empty dict, all metadata will be removed
f1.write_text("no metadata this time", metadata={})
print(f1.metadata)
print(f1.read_text())

## Use search to find data
You can search all the data that you have access to via key-value pairs.

Remember permissions are applied to search results, so you'll only see results to which you have at least read access.

Available arguments to `search` method:
- metadata: dictionary of key-value pairs that must match
- namespace: name of namespace to filter results
- dataset: name of dataset to filter results (namespace must be set along with dataset to be valid)
- modified_before: datetime string format `2006-01-02T15:04:05.000Z` to filter results
- modified_after: datetime string format `2006-01-02T15:04:05.000Z` to filter results
- limit: number of items to return per page
- offset: starting point in the index for returned items

In [ ]:
f2= ds / "with-meta"/ "test2.txt"
f2.write_text("my data file", metadata={'foo': 'bar', 'fizz': 'buzz'})
f3 = ds / "with-meta"/ "test3.txt"
f3.write_text("my data file", metadata={'foo': 'bar', 'fizz': 'other'})
f4 = ds / "with-meta"/ "test4.txt"
f4.write_text("my data file", metadata={'foo': 'bar'})
time.sleep(2)

In [ ]:
server_local.search({'foo': 'bar'})

In [ ]:
server_local.search({'foo': 'bar'}, limit=1)

In [ ]:
server_local.search({'foo': 'bar'}, limit=1, offset=1)

Using the `search_refs()` function will return `DatasetRef` instances instead of raw response data. This is useful because you can then directly interact with the results

In [ ]:
results = server_local.search_refs({'fizz': 'buzz'})
results

In [ ]:
results[0].read_text()

## Get suggestions for metadata keys and values
You can get suggestions for keys or values based on the existing metadata within a dataset.

Available arguments to the `suggest` methods:
- key: for value suggestions only, a key must be specified to filter suggestions.
- prefix: string prefix to base suggestions on. Suggestions will include keys or values beginning with this prefix.
- limit: number of objects to return suggestions. The exact number of suggestions may vary due to de-duplication of results and multiple matching keys or values within a single object.

In [ ]:
ds.suggest_keys('f')

In [ ]:
ds.suggest_values('fizz', 'bu')

## Clean up this example

Run these cells to remove the resources created during the test

In [ ]:
ns.delete_dataset("simple-test")